In [2]:
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git

Cloning into 'stylegan2-ada-pytorch'...
remote: Enumerating objects: 131, done.
remote: Counting objects: 100% (2/2), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 131 (delta 0), reused 0 (delta 0), pack-reused 129 (from 2)
Receiving objects: 100% (131/131), 1.13 MiB | 8.45 MiB/s, done.
Resolving deltas: 100% (57/57), done.


In [3]:
%cd stylegan2-ada-pytorch


/kaggle/working/stylegan2-ada-pytorch


Tính FID

In [ ]:
!ls /kaggle/input/fashion-product-images-dataset/fashion-dataset/images

In [9]:
import os
from PIL import Image
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import pandas as pd

df = pd.read_csv('/kaggle/input/fashion-dataset/val_fixed.csv',
                 on_bad_lines='skip')

src_dir = "/kaggle/input/fashion-product-images-dataset/fashion-dataset/images"
dst_dir = "/kaggle/working/val_images"
os.makedirs(dst_dir, exist_ok=True)

# Lấy tất cả id trong CSV → dạng string
csv_ids = set(df["image"].astype(str))

# Map ảnh trong thư mục theo id
files = []
for f in os.listdir(src_dir):
    name = f 
    if name in csv_ids:
        files.append(f)

print("Số ảnh match CSV:", len(files))

def process(file):
    try:
        src_path = os.path.join(src_dir, file)
        dst_path = os.path.join(dst_dir, file)
        img = Image.open(src_path).convert("RGB")
        img = img.resize((256, 256), Image.LANCZOS)
        img.save(dst_path)
    except Exception as e:
        print(f"Error processing {file}: {e}")

with ThreadPoolExecutor(max_workers=8) as executor:
    list(tqdm(executor.map(process, files), total=len(files)))

print("Done!")


Số ảnh match CSV: 4479


100%|██████████| 4479/4479 [02:02<00:00, 36.48it/s]

Done!


In [15]:
!mkdir /kaggle/working/models

In [11]:
!ls /kaggle/input/network-snapshot-004000-pkl

pytorch


In [18]:
!cp /kaggle/input/stylegan2-700kimg/pytorch/default/1/network-snapshot-000700.pkl /kaggle/working/models/network-snapshot-700.pkl
!cp /kaggle/input/stylegan2-1500kimg/pytorch/default/1/network-snapshot-001500-batch16.pkl /kaggle/working/models/network-snapshot-1500.pkl
!cp /kaggle/input/stylegan2-2300kimg/pytorch/default/1/network-snapshot-002300-noada.pkl  /kaggle/working/models/network-snapshot-2300.pkl
!cp /kaggle/input/stylegan2-3200kimg/pytorch/default/1/network-snapshot-003200.pkl       /kaggle/working/models/network-snapshot-3200.pkl
!cp /kaggle/input/network-snapshot-004000-pkl/pytorch/default/1/network-snapshot-004000.pkl        /kaggle/working/models/network-snapshot-4000.pkl

In [17]:
!python dataset_tool.py --source=/kaggle/working/val_images --dest=./datasets/fashion_image.zip

  0%|                                                  | 0/4479 [00:00<?, ?it/s]/kaggle/working/stylegan2-ada-pytorch/dataset_tool.py:429: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  img = PIL.Image.fromarray(img, { 1: 'L', 3: 'RGB' }[channels])
100%|██████████████████████████████████████| 4479/4479 [00:16<00:00, 264.49it/s]


In [20]:
!python calc_metrics.py --metrics=fid50k_full --data=./datasets/fashion_image.zip --mirror=1 \
    --network=/kaggle/working/models/network-snapshot-4000.pkl

Loading network from "/kaggle/working/models/network-snapshot-4000.pkl"...
Dataset options:
{
  "class_name": "training.dataset.ImageFolderDataset",
  "path": "./datasets/fashion_image.zip",
  "resolution": 256,
  "use_labels": false,
  "xflip": true
}
Launching processes...
Setting up PyTorch plugin "bias_act_plugin"... /usr/local/lib/python3.11/dist-packages/torch/utils/cpp_extension.py:2059: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... /usr/local/lib/python3.11/dist-packages/torch/utils/cpp_extension.py:2059: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
Done.

Generator             Parameters  Buffers  Output shape        Datatype
---     